# Command Line Tools

Aviary has a number of command line tools that are available via the `aviary`
command.

```{note}
When using a command line tool on a script that takes its own command line arguments, those
arguments must be placed after a `--` on the command line.  Anything to the right of the
`--` will be ignored by the Aviary command line parser and passed on to the user script.
For example: `Aviary sub_command -o foo.html myscript.py -- -x --myarg=bar` would pass
`-x` and `--myarg=bar` as args to `myscript.py`.
```

All available `aviary` sub-commands can be shown using the following command:

```
aviary -h
```

In [ ]:
!aviary -h

To get further info on any sub-command, follow the command with a *-h*.  For example:

```
aviary run_mission -h
```

In [ ]:
!aviary run_mission -h

## Available Commands and Their Usage


(aviary-run_mission-command)=
### aviary run_mission

`run_mission` is a command line interface that will run an analysis on a given csv input file.

To use small_single_aisle_GwGm.csv run the command `aviary run_mission models/small_single_aisle/small_single_aisle_GwGm.csv`<br>

SNOPT is the default optimizer, but IPOPT is available as well.

In [ ]:
# Testing Cell
import subprocess
command = 'aviary run_mission models/small_single_aisle/small_single_aisle_GwGm.csv'
command += ' --max_iter 0 --optimizer IPOPT' # max_iter to limit build time, IPOPT to run on CI
subprocess.run(command.split()).check_returncode();


```
aviary run_mission -h
```

In [ ]:
!aviary run_mission -h

(aviary-fortran_to_aviary-command)=
### aviary fortran_to_aviary

The `aviary fortran_to_aviary` command will convert a Fortran input deck to an Aviary csv.

The only required input is the filepath to the input deck.
Optionally, a deck of default values can be specified, this is useful if an input deck assumes certain values for any unspecified variables.
If an invalid filepath is given, pre-packaged resources will be checked for input decks with a matching name.

Notes for input decks:
- FLOPS, GASP, or Aviary names can be used for variables (Ex WG or Mission:Design:GROSS_MASS)
- When specifying variables from FORTRAN, they should be in the appropriate NAMELIST.
- Aviary variable names should be specified outside any NAMELISTS.
- Names are not case-sensitive.
- Units can be specified using any of the openMDAO valid units.
- Comments can be added using !
- Lists can be entered by separating values with commas.
- Individual list elements can be specified by adding an index after the variable name.
- (NOTE: 1 indexing is used inside NAMELISTS, while 0 indexing is used outside NAMELISTS)

Example inputs:
```
aircraft:fuselage:pressure_differential = .5, atm !DELP in GASP, but using atmospheres instead of psi
ARNGE(1) = 3600 !target range in nautical miles
pyc_phases = taxi, groundroll, rotation, landing
```


```
aviary fortran_to_aviary -h
```

In [ ]:
!aviary fortran_to_aviary -h

(aviary-hangar-command)=
### aviary hangar

The `aviary hangar` command will copy files and folders from the Aviary hangar to an accessible directory.
This is useful for users that have pip installed Aviary, but want to experiment with the included examples.

The only required input is the name of an input deck.
This can be specified as the name of the file, the path from [aviary/models](https://github.com/OpenMDAO/Aviary/tree/main/aviary/models), the name of a folder in aviary/models. Multiple files and folders can be copied at once.
Optionally, the output directory can be specified; if it is not, the files will be copied into the current working directory in a folder called aviary_models.
If specified, the output directory will be created as needed.

Example usage:
```
`aviary hangar engines` Copy all files in the engines folder
`aviary hangar turbofan_22k.txt` Copy the 22k turbofan deck
`aviary hangar N3CC/N3CC_data.py` Copy the N3CC data
`aviary hangar small_single_aisle_GwGm.dat small_single_aisle_GwGm.csv` Copy the Fortran and Aviary input decks for the small single aisle
`aviary hangar turbofan_22k.txt -o ~/example_files` Copy the engine model into ~/example_files
```


In [ ]:
# Testing Cell
import subprocess
import tempfile
import os
commands = [
    'engines','turbofan_22k.txt','N3CC/N3CC_data.py',
    'small_single_aisle_GwGm.dat small_single_aisle_GwGm.csv',
    'turbofan_22k.txt -o ~/example_files']
with tempfile.TemporaryDirectory() as tempdir:
    os.chdir(tempdir)
    for command in commands:
        command = 'aviary hangar '+command
        subprocess.run(command.split()).check_returncode();

```
aviary hangar -h
```

In [ ]:
!aviary hangar -h

(aviary-EDC-command)=
### aviary convert_engine

The `aviary convert_engine` command will convert a legacy formatted (FLOPS or GASP) engine deck into an Aviary formatted engine deck.

Users must provide the path or name of an input deck, the path to the output file, and the engine format being converted.
If an invalid filepath is given for the input file, pre-packaged resources will be checked for input decks with a matching name.

Notes for input decks:
- TurboFan decks for both FLOPS and GASP can be converted
- TurboProp decks for GASP can also be converted
- Comments can be added using #


Example usage:
```
`aviary convert_engine turbofan_23k_1.eng turbofan_23k_1_lbm_s.deck -f GASP` Convert a GASP based turbofan
`aviary convert_engine -f FLOPS turbofan_22k.eng turbofan_22k.txt` Convert a FLOPS based turbofan
`aviary convert_engine turboprop_4465hp.eng turboprop_4465hp.deck --data_format GASP_TP` Convert a GASP based turboprop
```


In [ ]:
# Testing Cell
from aviary.docs.tests.utils import run_command_no_file_error
commands = [
    'turbofan_23k_1.eng turbofan_23k_1_lbm_s.deck -f GASP',
    'turbofan_22k.eng turbofan_22k.txt -f FLOPS',
    'turboprop_4465hp.eng turboprop_4465hp.deck -f GASP_TP',
    ]
for command in commands:
    run_command_no_file_error('aviary convert_engine '+command)

```
aviary convert_engine -h
```

In [ ]:
!aviary convert_engine -h

(aviary-ATC-command)=
### aviary convert_aero_table

The `aviary convert_aero_table` command will convert a legacy formatted (FLOPS or GASP) aero table into an Aviary formatted aero table.

Users must provide the path or name of an input deck and the data format being converted.
Optionally, the path to the output file can also be specified, otherwise the default output file will be in the same location and have the same name as input file, but with '_aviary' appended to filename.
If an invalid filepath is given for the input file, pre-packaged resources will be checked for input decks with a matching name.

Notes for input decks:
- Aero tables for both FLOPS and GASP can be converted
- GASP tables will create a single file containing all the lift and drag information
- FLOPS tables will create two files, one containing the lift-dependent drag and the other containing the lift-independent drag.
- Comments can be added using #


Example usage:
```
`aviary convert_aero_table -f GASP subsystems/aerodynamics/gasp_based/data/GASP_aero_free.txt large_single_aisle_1_aero_flaps.txt` Convert a GASP based aero table
`aviary convert_aero_table -f FLOPS utils/test/flops_test_polar.txt aviary_flops_polar.txt` Convert a FLOPS based aero table
```


In [ ]:
# Testing Cell
from aviary.docs.tests.utils import run_command_no_file_error
commands = [
    '-f GASP subsystems/aerodynamics/gasp_based/data/GASP_aero_free.txt large_single_aisle_1_aero_flaps.txt',
    '-f FLOPS utils/test/flops_test_polar.txt aviary_flops_polar.txt',
]
for command in commands:
    run_command_no_file_error('aviary convert_aero_table '+command)


```
aviary convert_aero_table -h
```

In [ ]:
!aviary convert_aero_table -h

(aviary-PMC-command)=
### aviary convert_prop_table

The `aviary convert_prop_table` command will convert a legacy formatted (GASP) propeller map into an Aviary formatted propeller map.

Users must provide the path or name of an input deck, the path to the output file, and the engine format being converted.
If an invalid filepath is given for the input file, pre-packaged resources will be checked for input decks with a matching name.

Notes for input decks:
- There are two options for Mach number: Mach number and helical Mach number at 75% radius. We provide one example for each. This is the first integer on the first line of the input file. (1 = Mach, 2 = helical Mach)
- Comments can be added using #


Example usage:
```
`aviary convert_prop_table -f GASP PropFan.map PropFan.prop` Convert a GASP based propeller map
`aviary convert_prop_table -f GASP general_aviation.map general_aviation.prop` Convert a GASP based propeller map
```

The first example uses Mach number and the second example uses helical Mach number. 
Note that the output file name can be skipped. Since there is only one input data format is supported at this time, input data format is defaulted to GASP if not provided.

```
aviary convert_prop_table -h
```

In [ ]:
!aviary convert_prop_table -h